In [1]:
%%capture
import os
if "COLAB_" not in "".join(os.environ.keys()):
    !pip install unsloth
else:
    # Do this only in Colab notebooks! Otherwise use pip install unsloth
    !pip install --no-deps bitsandbytes accelerate xformers==0.0.29.post3 peft trl triton cut_cross_entropy unsloth_zoo
    !pip install sentencepiece protobuf "datasets>=3.4.1" huggingface_hub hf_transfer
    !pip install --no-deps unsloth

!pip install gTTS
!apt-get install -y portaudio19-dev
!pip install sounddevice scipy speechrecognition
!apt-get install -y ffmpeg
!pip install openai-whisper
!pip install pyngrok

# Importing Libraries

In [25]:
from unsloth import FastVisionModel
import torch
from PIL import Image
import requests
from io import BytesIO
import io
from IPython.display import display, Javascript
from google.colab import output
import base64
from pydub import AudioSegment
import whisper
from gtts import gTTS
from IPython.display import Audio
from fastapi import FastAPI, UploadFile, File
from fastapi.responses import JSONResponse
import nest_asyncio, asyncio
import uvicorn
from pyngrok import ngrok
from collections import deque
from transformers import TextStreamer
import gc

<h1>VLM (Llama-3.2-11B-Vision-Instruct)</h1>

In [3]:
vlm_model, tokenizer = FastVisionModel.from_pretrained(
    "unsloth/Llama-3.2-11B-Vision-Instruct",
    load_in_4bit = True,
    use_gradient_checkpointing = "unsloth",
)

==((====))==  Unsloth 2025.6.12: Fast Mllama patching. Transformers: 4.53.0.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.6.0+cu124. CUDA: 7.5. CUDA Toolkit: 12.4. Triton: 3.2.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.29.post3. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [4]:
vlm_model = FastVisionModel.get_peft_model(
    vlm_model,
    finetune_vision_layers     = True,
    finetune_language_layers   = True,
    finetune_attention_modules = True,
    finetune_mlp_modules       = True,

    r = 16,
    lora_alpha = 16,
    lora_dropout = 0,
    bias = "none",
    random_state = 3407,
    use_rslora = False,
    loftq_config = None,
)

Unsloth: Making `model.base_model.model.model.vision_model.transformer` require gradients


In [ ]:
# Image URL
image_url = 'https://huggingface.co/datasets/huggingface/documentation-images/resolve/main/p-blog/candy.JPG'

# Load and display the image
response = requests.get(image_url)
image = Image.open(BytesIO(response.content))
image

In [ ]:
FastVisionModel.for_inference(model) # Enable for inference!
instruction = "What animal is on the candy?"

messages = [
    {"role": "user", "content": [
        {"type": "image"},
        {"type": "text", "text": instruction}
    ]}
]
input_text = tokenizer.apply_chat_template(messages, add_generation_prompt = True)
inputs = tokenizer(
    image,
    input_text,
    add_special_tokens = False,
    return_tensors = "pt",
).to("cuda")

from transformers import TextStreamer
text_streamer = TextStreamer(tokenizer, skip_prompt = True)
_ = vlm_model.generate(**inputs, streamer = text_streamer, max_new_tokens = 128,
                   use_cache = True, temperature = 1.5, min_p = 0.1)

The animal on the candy appears to be a turtle based on the outline and shape, specifically the sea turtle species, which is the state animal for Florida.<|eot_id|>


In [ ]:
image_url = '/content/IMG_6812.jpeg'

# Load and display the image
image = Image.open(image_url)
image

In [ ]:
instruction = "What is on the Laptop and what is on the monitor?"

messages = [
    {"role": "user", "content": [
        {"type": "image"},
        {"type": "text", "text": instruction}
    ]}
]
input_text = tokenizer.apply_chat_template(messages, add_generation_prompt = True)
inputs = tokenizer(
    image,
    input_text,
    add_special_tokens = False,
    return_tensors = "pt",
).to("cuda")

from transformers import TextStreamer
text_streamer = TextStreamer(tokenizer, skip_prompt = True)
_ = vlm_model.generate(**inputs, streamer = text_streamer, max_new_tokens = 128,
                   use_cache = True, temperature = 1.5, min_p = 0.1)

On the laptop, the news article is an article from Hollywood Reporter by Samuel Acosta, with the title “The Batman” Shows off the World’s Greatest Detective.

The monitor shows the site “Murphy’s Multiverse” with a review of the movie “The Amazing Spiderman 2,” which mentions it being a great movie, but “you ignore a lot of it,” possibly in regards to the fact that some parts were cut out or simplified from the original storyline.

Note: The actual image doesn't state exactly why he thinks the parts were “ignored”, though in the 2002 Spider-Man there are 10 villains, and


In [ ]:
instruction = "What are the exact main headlines on both laptop and monitor screens?"

messages = [
    {"role": "user", "content": [
        {"type": "image"},
        {"type": "text", "text": instruction}
    ]}
]
input_text = tokenizer.apply_chat_template(messages, add_generation_prompt = True)
inputs = tokenizer(
    image,
    input_text,
    add_special_tokens = False,
    return_tensors = "pt",
).to("cuda")

from transformers import TextStreamer
text_streamer = TextStreamer(tokenizer, skip_prompt = True)
_ = vlm_model.generate(**inputs, streamer = text_streamer, max_new_tokens = 128,
                   use_cache = True, temperature = 1.5, min_p = 0.1)

The laptop screen has a headline that reads: “‘The Batman’ Shows Off the World's Greatest Detective”. The screen has no visible headlines or text.

The monitor has a headline that reads: “REVIEW: ‘The Amazing Spider-Man 2’ Is Great If You Ignore A Lot Of It”.<|eot_id|>


<h1>Speech-To-Text (STT)</h1>

In [6]:
audio_data = None

def receive_audio(base64data):
    global audio_data
    header, data = base64data.split(',', 1)
    audio_bytes = base64.b64decode(data)
    audio_data = io.BytesIO(audio_bytes)
    print("Audio recorded and received!")

    # Convert to WAV and save
    audio_segment = AudioSegment.from_file(audio_data)
    audio_segment.export("recorded.wav", format="wav")
    print("saved as recorded.wav")

output.register_callback('notebook.ReceiveAudio', receive_audio)

def record(sec=5):
    display(Javascript(f'''
    async function record(sec) {{
      const stream = await navigator.mediaDevices.getUserMedia({{audio:true}});
      const mediaRecorder = new MediaRecorder(stream);
      let audioChunks = [];
      mediaRecorder.ondataavailable = e => audioChunks.push(e.data);
      mediaRecorder.start();
      await new Promise(resolve => setTimeout(resolve, sec*1000));
      mediaRecorder.stop();
      await new Promise(resolve => mediaRecorder.onstop = resolve);
      const audioBlob = new Blob(audioChunks);
      const reader = new FileReader();
      reader.readAsDataURL(audioBlob);
      reader.onloadend = () => {{
        const base64data = reader.result;
        google.colab.kernel.invokeFunction('notebook.ReceiveAudio', [base64data], {{}});
      }}
    }}
    record({sec});
    '''))

# Start recording
record(5)

<IPython.core.display.Javascript object>

Audio recorded and received!
saved as recorded.wav


In [7]:
whisper_model = whisper.load_model("base")

100%|████████████████████████████████████████| 139M/139M [00:01<00:00, 113MiB/s]


In [10]:
result = whisper_model.transcribe("recorded.wav")
print(result["text"])

 Is there a pen on the table?


<h1>Text-To-Speech (TTS)</h1>

In [ ]:
# Generate the speech
tts = gTTS(text="Hello, this is a test one. \n I am a Master's student at UB university.", lang='en')
tts.save("output.mp3")

In [ ]:
Audio("output.mp3", autoplay=True)

In [12]:
Audio("recorded.wav", autoplay=True)

<h1>Converting mp3 to wav in case to feed the generated audio to process in speechRecognition library and get text back</h1>

In [ ]:
# !pip install pydub
# !apt-get install ffmpeg -y

In [ ]:
# from pydub import AudioSegment

# # Load MP3
# audio = AudioSegment.from_mp3("recorded.mp3")

# # Export to WAV with correct format (mono, 16kHz, PCM)
# audio = audio.set_channels(1).set_frame_rate(16000)
# audio.export("output.wav", format="wav")

#Endpoint Creation using Flask

In [13]:
class VLMHandler():
    def initialize(self):
        self.model, self.tokenizer = vlm_model, tokenizer
        self.model.eval()

    def preprocess(self, image_url):
        response = requests.get(image_url)
        image = Image.open(BytesIO(response.content)).convert("RGB")
        return image

    def inference(self, images, instruction):
        messages = [
            {"role": "user", "content": (
                [{"type": "image"} for _ in images] + [{"type": "text", "text": instruction}]
            )}
        ]
        input_text = self.tokenizer.apply_chat_template(messages, add_generation_prompt=True)

        inputs = self.tokenizer(
            images,
            input_text,
            add_special_tokens=False,
            return_tensors="pt",
        ).to("cuda")

        text_streamer = TextStreamer(self.tokenizer, skip_prompt=True)
        with torch.no_grad():
            outputs = self.model.generate(
                **inputs,
                streamer=text_streamer,
                max_new_tokens=50,
                use_cache=True,
                temperature=1.5,
                min_p=0.1
            )
        text_output = self.tokenizer.decode(outputs[0], skip_special_tokens=True)

        # Cleanup
        del inputs, outputs, text_streamer
        torch.cuda.empty_cache()
        gc.collect()

        return text_output

    def postprocess(self, inference_output):
        return inference_output.split('\n')[-1]

In [14]:
vlm_handler = VLMHandler()
vlm_handler.initialize()

In [15]:
image  = vlm_handler.preprocess('https://huggingface.co/datasets/huggingface/documentation-images/resolve/main/p-blog/candy.JPG')
instruction = "What animal is on the candy? Just answer in one word."
inference_output = vlm_handler.inference([image], instruction)
final_output = vlm_handler.postprocess(inference_output)

Turtle.<|eot_id|>


In [16]:
print(final_output)

Turtle.


In [30]:
app = FastAPI()
queue = deque(maxlen=7)

@app.get("/")
async def home():
    return {"message": "VLMATE!!"}

@app.get("/getQcount")
async def getQcount():
    return {
        "image_queue_size": len(queue),
    }

@app.post("/storeImage")
async def storeImage(image: UploadFile = File(...)):
    contents = await image.read()
    pil_image = Image.open(BytesIO(contents)).convert("RGB")
    queue.append(pil_image)
    return {"message": "Image stored successfully", "queue_size": len(queue)}

@app.post("/audioPredict")
async def audioPredict(audio: UploadFile = File(...)):
    contents = await audio.read()

    with open("temp_audio_file.mp3", "wb") as f:
        f.write(contents)

    result = whisper_model.transcribe("temp_audio_file.mp3")
    transcription = result["text"]

    if not queue:
        return JSONResponse(status_code=400, content={"error": "No images in queue"})

    frames_in_Q = list(queue)

    try:
        loop = asyncio.get_running_loop()
        outputs = await loop.run_in_executor(None, vlm_handler.inference, frames_in_Q, transcription)
        result_text = vlm_handler.postprocess(outputs)
        return {
            "transcription": transcription,
            "vlm_result": result_text
        }
    except Exception as e:
        return JSONResponse(status_code=500, content={"error": str(e)})

@app.post("/predict")
async def predict(instruction: str = "Describe this image."):
    if not queue:
        return JSONResponse(status_code=400, content={"error": "No images in queue"})

    frames_in_Q = list(queue)

    try:
        loop = asyncio.get_running_loop()
        outputs = await loop.run_in_executor(None, vlm_handler.inference, frames_in_Q, instruction)
        result_text = vlm_handler.postprocess(outputs)
        return {"result": result_text}
    except Exception as e:
        return JSONResponse(status_code=500, content={"error": str(e)})

In [27]:
from google.colab import userdata
ngrok_token = userdata.get('ngrok_auth')

In [28]:
!ngrok config add-authtoken {ngrok_token}

Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml


In [ ]:
public_url = ngrok.connect(5000)
print("Your app is live at:", public_url)

nest_asyncio.apply()
uvicorn.run(app, host="0.0.0.0", port=5000)

Your app is live at: NgrokTunnel: "https://a1e8-34-125-156-232.ngrok-free.app" -> "http://localhost:5000"


INFO:     Started server process [12435]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://0.0.0.0:5000 (Press CTRL+C to quit)


INFO:     2603:7081:76f0:6310:38f0:ddf9:7c75:6397:0 - "GET / HTTP/1.1" 200 OK
INFO:     2603:7081:76f0:6310:38f0:ddf9:7c75:6397:0 - "GET /favicon.ico HTTP/1.1" 404 Not Found
INFO:     2603:7081:76f0:6310:38f0:ddf9:7c75:6397:0 - "GET / HTTP/1.1" 200 OK
INFO:     2603:7081:76f0:6310:38f0:ddf9:7c75:6397:0 - "GET /getQcount HTTP/1.1" 200 OK
INFO:     2603:7081:76f0:6310:38f0:ddf9:7c75:6397:0 - "POST /storeImage HTTP/1.1" 200 OK


ERROR:asyncio:Task exception was never retrieved
future: <Task finished name='Task-9' coro=<Server.serve() done, defined at /usr/local/lib/python3.11/dist-packages/uvicorn/server.py:69> exception=KeyboardInterrupt()>
Traceback (most recent call last):
  File "/usr/local/lib/python3.11/dist-packages/uvicorn/main.py", line 580, in run
    server.run()
  File "/usr/local/lib/python3.11/dist-packages/uvicorn/server.py", line 67, in run
    return asyncio.run(self.serve(sockets=sockets))
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/nest_asyncio.py", line 30, in run
    return loop.run_until_complete(task)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/nest_asyncio.py", line 92, in run_until_complete
    self._run_once()
  File "/usr/local/lib/python3.11/dist-packages/nest_asyncio.py", line 133, in _run_once
    handle._run()
  File "/usr/lib/python3.11/asyncio/events.py", line 84, in _run
    se

The image shows a desk with a computer, notebook, and colored pencils. The purpose of the image is to show a workspace where someone is working on a project.

*   **Computer:**
    *   The computer is an Apple MacBook laptop with


ERROR:asyncio:Task exception was never retrieved
future: <Task finished name='Task-1' coro=<Server.serve() done, defined at /usr/local/lib/python3.11/dist-packages/uvicorn/server.py:69> exception=KeyboardInterrupt()>
Traceback (most recent call last):
  File "/usr/local/lib/python3.11/dist-packages/uvicorn/main.py", line 580, in run
    server.run()
  File "/usr/local/lib/python3.11/dist-packages/uvicorn/server.py", line 67, in run
    return asyncio.run(self.serve(sockets=sockets))
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/nest_asyncio.py", line 30, in run
    return loop.run_until_complete(task)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/nest_asyncio.py", line 92, in run_until_complete
    self._run_once()
  File "/usr/local/lib/python3.11/dist-packages/nest_asyncio.py", line 133, in _run_once
    handle._run()
  File "/usr/lib/python3.11/asyncio/events.py", line 84, in _run
    se

INFO:     2603:7081:76f0:6310:38f0:ddf9:7c75:6397:0 - "POST /predict HTTP/1.1" 200 OK
Yes, there is a blue pen on the table.  The pen is lying on top of a spiral-bound notebook.<|eot_id|>
INFO:     2603:7081:76f0:6310:38f0:ddf9:7c75:6397:0 - "POST /audioPredict HTTP/1.1" 200 OK


Home [GET]:

```bash
curl -X GET https://abc.ngrok-free.app \
 -H "Content-Type: application/json"


get queue count [GET]:

```bash
curl -X GET https://abc.ngrok-free.app/getQcount \
 -H "Content-Type: application/json"


store the images [POST]:

```bash
curl -X POST https:/abc.ngrok-free.app/storeImage \
 -F "image=@/Users/phanitarunmunukuntla/Downloads/content/frames/frame_0000.jpg"


text input - vlm [POST]:

```bash
curl -X POST https:/abc.ngrok-free.app/predict \
 -H "Content-Type: application/json" \
 -d '{"instruction": "Describe this image."}'


audio input - vlm [POST]:

```bash
curl -X POST https://abc123.ngrok.io/audioPredict \
  -F "audio=@/path/to/your/audiofile.mp3"
  

In [ ]:

'''

curl -X GET https://a1e8-34-125-156-232.ngrok-free.app \
 -H "Content-Type: application/json"

curl -X GET https://a1e8-34-125-156-232.ngrok-free.app/getQcount \
 -H "Content-Type: application/json"

curl -X POST https://a1e8-34-125-156-232.ngrok-free.app/storeImage \
  -F "image=@/Users/phanitarunmunukuntla/Downloads/content/frames/frame_0000.jpg"

curl -X POST https://a1e8-34-125-156-232.ngrok-free.app/predict \
  -H "Content-Type: application/json" \
  -d '{"instruction": "Do you see any pen or pencil on the table?"}'

curl -X POST https://a1e8-34-125-156-232.ngrok-free.app/audioPredict \
  -F "audio=@/Users/phanitarunmunukuntla/Downloads/recorded.mp3"


 '''